In [36]:
import sys
import os
sys.path.append("/mnt/c/Users/bach484/Documents/EcoliTRN-ModuleEval/src/modeval/")

import json
import shutil

from util import JSONExtendedEncoder

from clustering import *
from biclustering import *
from moduleni import *
from directni import *

import pandas as pd
import numpy as np

import time

import traceback
from modulecontainers import Modules

%load_ext autoreload
%autoreload 2

%matplotlib inline
from matplotlib.pyplot import *

import multiprocessing as mp

from collections import OrderedDict

from itertools import product
from collections import defaultdict

import itertools
import shutil

conf_folder = "conf/"

import os

from modulescomparison import ModevalKnownmodules, ModevalCoverage

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
settings = []
settings.append({
    "dataset_location":"conf/ecoli_dataset.json", 
    "dataset_name":"ecoli_precise2",
    "method_location":"conf/method_genie3.json", 
    "regnet_name":"ecoli_regulondb",
    "knownmodules_name":"minimal",
    "output_folder":"results/genie3/",
    "settingid": 1
    }
)

In [38]:
method_path = "../conf/method_genie3.json"
dataset_path = "../conf/ecoli_dataset.json"
output_folder = "../results/genie3/"

# Load the method and dataset JSON files
with open(method_path, "r") as file:
    method = json.load(file)

with open(dataset_path, "r") as file:
    dataset = json.load(file)

knownmodules = json.load(open(dataset["knownmodules"]["ecoli_regulondb"]["minimal"]))
knownmodules = pd.DataFrame(knownmodules)
regulators =json.load(open('../data/regulators.json'))

starttime = time.time()

try:
	modules = locals()[method["params"]["method"]](knownmodules, regulators, **method["params"])
except BaseException as e:
	print("Error during clustering: ")
	print(e)
	traceback.print_exc()
	modules = []

endtime = time.time()

if os.path.exists(output_folder):
	shutil.rmtree(output_folder)
os.makedirs(output_folder)
json.dump(modules, open(output_folder + "modules.json", "w"), cls=JSONExtendedEncoder)
json.dump({"runningtime":endtime-starttime}, open(output_folder + "runinfo.json", "w"), cls=JSONExtendedEncoder)
json.dump(method["params"], open(output_folder + "method.json", "w"))


24

        source("../methods/genie3_args.R")

        E = read.delim("/tmp/tmplki5n7qu/E.csv", header=T, row.names=1, sep=",")

        R = as.character(read.csv("/tmp/tmplki5n7qu/R.csv", header=F)[[1]])

        results = run.genie3(E, tf.idx=R, mc.cores=24)

        write.table(results, "/tmp/tmplki5n7qu/results.csv", sep="\t")

        
Error during clustering: 
[Errno 2] No such file or directory: '/tmp/tmplki5n7qu/results.csv'


Error in library(tidyverse) : there is no package called ‘tidyverse’
Calls: source -> withVisible -> eval -> eval -> library
Execution halted
Traceback (most recent call last):
  File "/tmp/ipykernel_28298/596245968.py", line 19, in <module>
    modules = locals()[method["params"]["method"]](knownmodules, regulators, **method["params"])
  File "/mnt/c/Users/bach484/Documents/EcoliTRN-ModuleEval/src/modeval/directni.py", line 56, in genie3
    wnet = pd.read_csv(tmpdir + "/results.csv", sep="\t")
  File "/home/bach484/miniconda3/envs/project/lib/python3.9/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/home/bach484/miniconda3/envs/project/lib/python3.9/site-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/home/bach484/miniconda3/envs/project/lib/python3.9/site-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = s

In [39]:
# create pool of processors
if "pool" in locals().keys():
    pool.close()
pool = mp.Pool(mp.cpu_count()-1)

In [40]:
modeval = ModevalKnownmodules(settings, baseline = True)
modeval.run(pool)

In [41]:
scores = []
for setting in settings:
    score = modeval.scores.loc[
        (modeval.scores["settingid"] == setting["settingid"]) &
        (modeval.scores["knownmodules_name"] == setting["knownmodules_name"]) &
        (modeval.scores["regnet_name"] == setting["regnet_name"])
    ]
    
scores.append(score)
scores = pd.concat(scores)
scores["baseline_name"] = "genie3"

In [42]:
scores

,recovery,relevance,F1rr,recall,precision,F1rp,F1rprr,F1rr_permuted,F1rp_permuted,F1rprr_permuted,settingid,knownmodules_name,regnet_name,goldstandard,runningtime,baseline_name
0,0.0,0.0,0,0.0,0.0,0,0,0,0,0,1,minimal,ecoli_regulondb,ecoli_regulondb#minimal,0.164151,genie3
